# Practical session: Convolutional Neural Networks (CNN) 

Convolutional Neural Networks (CNNs) are a type of deep learning model particularly effective for processing and analyzing visual data like images and videos. CNNs use a special structure that allows them to automatically and adaptively learn spatial hierarchies of features. This is done through layers like convolutional layers, pooling layers, and fully connected layers. Convolutional layers apply filters to the input, capturing essential patterns such as edges, textures, and shapes. This makes CNNs especially powerful for tasks like image classification, object detection, and facial recognition.

<img width="600" alt="image" src="https://saturncloud.io/images/blog/a-comprehensive-guide-to-convolutional-neural-networks-the-eli5-way.webp">

## Overview

This practical session focuses on using Convolutional Neural Networks (CNNs) to classify images into ten distinct categories, utilizing the `Fashion-MNIST` dataset.

### Key Objectives:
- **Grasping the Fundamentals of CNNs:** Gain a solid understanding of how CNNs function, including their role in image classification tasks.
- **Exploring Layer Functionality:** Learn how different layers in a CNN contribute to feature extraction and classification, transforming raw image data into meaningful predictions.
- **Constructing a Basic CNN Model:** Build a simple yet effective CNN model tailored to the Fashion-MNIST dataset, applying best practices in model architecture.
- **Training the CNN Model:** Train the CNN model, understanding the role of epochs, batch sizes, and the learning process involved in optimizing the model’s performance.
- **Evaluating Model Performance:** Assess the accuracy and effectiveness of your CNN model using appropriate evaluation metrics and visualization techniques.
- **Tuning CNN Hyperparameters:** Experiment with and manipulate key hyperparameters to improve model performance and gain deeper insights into CNN optimization.


## Dataset

`Fashion-MNIST` is a dataset of Zalando's article images—consisting of a training set of 60,000 examples and a test set of 10,000 examples. Each example is a 28x28 grayscale image, associated with a label from 10 classes. 

<img width="400" alt="image" src="https://i.imgur.com/BxGMWW4.png">


You can use direct links to download the dataset or use the folder `/data`.

- Original source: https://github.com/zalandoresearch/fashion-mnist

Folder `/data` includes a column based format of this dataset: 
- `fashion-mnist_train.csv`
- `fashion-mnist_test.csv`

A quick description of these files: 

- Each image measures 28 by 28 pixels, totaling 784 pixels.
- Every pixel has a corresponding value that reflects its brightness, with higher values indicating a darker shade. These values range from 0 to 255.
- Both the training and test datasets contain 785 columns.
- The first column holds the class labels, representing the type of clothing item.
- The remaining 784 columns (1-785) store the pixel values for the corresponding image.


### Labels
Each training and test example is assigned to one of the following labels:

| Label | Description |
| --- | --- |
| 0 | T-shirt/top |
| 1 | Trouser |
| 2 | Pullover |
| 3 | Dress |
| 4 | Coat |
| 5 | Sandal |
| 6 | Shirt |
| 7 | Sneaker |
| 8 | Bag |
| 9 | Ankle boot |



### Data loading

In [ ]:
## Importing Libraries 

# Base libraries
import numpy as np
import pandas as pd
import os
import datetime
import warnings
warnings.filterwarnings('ignore')

# Visualisation 
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

# Models
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, ReLU
from tensorflow.keras.utils import plot_model
from tensorflow.keras.utils import to_categorical

print("tensorflow v."+tf.__version__)

## Default options and global variables
# Set number of decimal points to float type
pd.set_option("display.float_format", lambda x: "%.2f" % x)
pd.set_option('display.precision', 2)
SEED = 2024
N_CLASSES = 10
keras.utils.set_random_seed(SEED)


In [ ]:
# Set the data path 
DATA_PATH="../data/Fashion-MNIST/"

# Data loading
df_train = pd.read_csv(DATA_PATH + "fashion-mnist_train.csv.zip")
df_test = pd.read_csv(DATA_PATH + "fashion-mnist_test.csv.zip")

# Data dimension 
print("Train set:",df_train.shape[0],"rows,",  df_train.shape[1], "columns")
print("Test set :",df_test.shape[0],"rows,",  df_test.shape[1], "columns")


### Quick exploration

In [ ]:
# Quick exploration 
df_train.head()

### Target variable

In [ ]:
# Labels  
labels = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

def plot_label_distributions (data, info):
    plt.figure(figsize=(6,2))
    unique_labels, data_counts = np.unique(data, return_counts=True)
    sns.barplot(x=unique_labels, y=data_counts)
    # labeling the plot
    plt.xticks( unique_labels, labels, rotation=60)
    plt.ylabel('Count')
    plt.title('Distribution of each class - ' + info)

plot_label_distributions(df_train['label'], "train")
plot_label_distributions(df_test['label'], "test")


In [6]:
## Subsampling ~ 50% 
df_train_sample = df_train.sample(n=30000, random_state=SEED)

In [ ]:
plot_label_distributions(df_train_sample['label'], "Subsampled train")

## Modelling 

We use [tf.keras](https://www.tensorflow.org/guide/keras) to build and train models. Keras is the high-level API of the TensorFlow platform. It provides an approachable, highly-productive interface for solving machine learning (ML) problems, with a focus on modern deep learning.

The `tf.keras.Model` class features built-in training and evaluation methods:

- `tf.keras.Model.fit`: Trains the model for a fixed number of epochs.
- `tf.keras.Model.predict`: Generates output predictions for the input samples.
- `tf.keras.Model.evaluate`: Returns the loss and metrics values for the model; configured via the `tf.keras.Model.compile` method.

### Data preparation

In [8]:
## Dataset preparation 

# Variables for modelling
#x_train = df_train.drop(columns=['label']).values
#y_train = df_train[['label']].values
x_train = df_train_sample.drop(columns=['label']).values
y_train = df_train_sample[['label']].values
x_test = df_test.drop(columns=['label']).values
y_test = df_test[['label']].values

# Reshaping the colums 784 (one dimensional format) to 28x28 (two dimensional array)
x_train = x_train.reshape(-1, 28, 28, 1)
x_test  = x_test.reshape(-1, 28, 28, 1)

# Alternative, you can fetch the "Fashion MNIST" data from Keras. 
#(x_train, y_train), (x_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()

# Use OneHotEncoder to encode target column ~ binary representation 
y_train = to_categorical(y_train, num_classes=N_CLASSES)
y_test  = to_categorical(y_test, num_classes=N_CLASSES)


In [ ]:
print("Shape of Training Image Data: " + str(x_train.shape))
print("Shape of Training Class Data: " + str(y_train.shape))
print("Shape of Test Image Data: " + str(x_test.shape))
print("Shape of Test Class Data: " + str(y_test.shape))

In [ ]:
#np.argmax( y_train[2])
y_train[2]

In [ ]:
# plot few images 
plt.figure(figsize=(6,6))
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(x_train[i], cmap=plt.cm.binary)
    plt.xlabel( labels[np.argmax(y_train[i])])
plt.show()

### Model definition

In [13]:
## Model definition
model = Sequential() # The sequential model is a linear stack of layers. Add layers by using the `add` method 
# Layers 
model.add( Flatten())
model.add( Dense(128, activation='tanh'))
model.add( Dense(N_CLASSES, activation='softmax'))

### Model compilation

Before the model is ready for training, a few additional settings need to be configured during the model's compile step:

- **Loss Function**: Measures the model's accuracy during training. The goal is to minimize this loss, guiding the model to improve its predictions.
- **Optimizer**: Updates the model's parameters based on the data and loss. This helps the model learn from its errors.
- **Metrics**: Track the model's performance during training and testing.


In [14]:
## Model compliation 
optimizer = keras.optimizers.legacy.SGD(learning_rate=0.1)
model.compile(
    loss=keras.losses.categorical_crossentropy,
    optimizer= optimizer,
    metrics=['accuracy'])


In [ ]:
## Model vis
model.build(input_shape=(None, 28,28,1)) # needed when the input_shape is not defined
model.summary()
plot_model(model, show_shapes=True, show_layer_names=True)

### Model fitting


In [ ]:
BATCH=64
EPOCHS=12

history_model = model.fit(x_train, y_train,
                  batch_size=BATCH,
                  epochs=EPOCHS,
                  verbose=1)

### Model evaluation 

In [ ]:
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [ ]:
from matplotlib.ticker import MaxNLocator
#%matplotlib inline

def plot_model_history (history_model):
    fig, (ax2, ax1) = plt.subplots(nrows=1, ncols=2, figsize=(10, 3))
    # loss
    ax1.plot(history_model.history['loss'], label='Train Loss')
    ax1.xaxis.set_major_locator(MaxNLocator(integer=True))
    ax1.legend()
    # accuracy
    ax2.plot(history_model.history['accuracy'], label='Train Accuracy')
    ax2.xaxis.set_major_locator(MaxNLocator(integer=True))
    ax2.legend()

plot_model_history(history_model)

### Model predictions




In [ ]:
predictions = model.predict(x_test)

In [ ]:
# A prediction is an array of the confidence values that correspont to each category (label)
predictions[6]

In [ ]:
# Label with the highest confidence
np.argmax(predictions[6])

In [22]:
## Helper functions
def plot_prediction(i, predictions, true_label, img):
  predictions, true_label, img = predictions[i], true_label[i], img[i]
  plt.grid(False)
  plt.xticks([])
  plt.yticks([])
  
  plt.imshow(img, cmap=plt.cm.binary)

  predicted_label = np.argmax(predictions)
  if predicted_label == true_label:
    color = 'blue'
  else:
    color = 'red'
  
  plt.xlabel("{} {:2.0f}% ({})".format(labels[predicted_label],
                                100*np.max(predictions),
                                labels[true_label]),
                                color=color)

def plot_prediction_confidence(i, predictions, true_label):
  predictions, true_label = predictions[i], true_label[i]
  plt.grid(False)
  #plt.xticks()
  plt.xticks( range(N_CLASSES), labels, rotation=80)
  plt.yticks([])
  thisplot = plt.bar(range(N_CLASSES), predictions, color="#777777")
  plt.ylim([0, 1]) 
  predicted_label = np.argmax(predictions)
 
  thisplot[predicted_label].set_color('red')
  thisplot[true_label].set_color('blue')

In [ ]:
i = 90
plt.figure(figsize=(6,3))
plt.subplot(1,2,1)
plot_prediction(i, predictions, df_test['label'], x_test)
plt.subplot(1,2,2)
plot_prediction_confidence(i, predictions,  df_test['label'])
plt.show()

### Model mistakes

In [ ]:
from sklearn.metrics import confusion_matrix
from itertools import product


##Create Multiclass Confusion Matrix
def plot_confusion_matrix(predictions, y_test):
    cm = confusion_matrix(np.argmax(y_test,axis=1), np.argmax(predictions,axis=1))

    plt.figure(figsize=(8,8))
    plt.imshow(cm,cmap=plt.cm.Blues)
    plt.title('Confusion Matrix - Fashion-MNIST')
    plt.colorbar()
    plt.xticks(range(N_CLASSES), labels, rotation=90)
    plt.yticks(range(N_CLASSES), labels)

    for i, j in product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
        horizontalalignment="center",
        color="white" if cm[i, j] > 500 else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

plot_confusion_matrix(predictions,y_test)

## Improving the model

...
